### to merge fmg and grist results

In [38]:
#!/usr/bin/env python3
import pandas as pd

# Input files
FMG = "./fmg/fmg_summary.csv"
GRIST = "./grist/grist_summary.csv"
FMAPPED = "./f_mapped_reads/f_mapped_reads.csv"
JOIN_G = "../analysis_rs226_k21/join_rs226_k21.csv"

# Load tables
fmg = pd.read_csv(FMG)
grist = pd.read_csv(GRIST)
fmap = pd.read_csv(FMAPPED)
join_g = pd.read_csv(JOIN_G)

fmg.head()

,sample_id,n_match_genomes_fmg,avg_f_match,avg_median_abund,n_unique_weighted
0,DRR014782,13557,0.019701,1.880910,658191
1,DRR014785,14609,0.020887,1.879697,730806
2,DRR019499,23740,0.006569,3.610088,169107
3,DRR042462,3328,0.027927,2.406250,140571
4,DRR042463,4517,0.020202,1.889750,123562


In [39]:
grist.head()

,sample_id,n_match_genomes,avg_f_covered_bp,avg_effective_coverage,n_mapped_reads
0,DRR014782,735,0.283430,5.906585,75612507
1,DRR014785,892,0.272454,5.303160,83163638
2,DRR121397,234,0.131079,1.655980,2121643
3,DRR121401,175,0.121921,1.409490,1311236
4,ERR10695318,57,0.265579,126.762150,123175020


In [40]:
fmap.head()

,sample_id,total_mapped_reads,total_n_reads,f_mapped_reads
0,DRR014782,75612507,83531900,0.905193
1,DRR014785,83163638,91931356,0.904628
2,DRR121397,2121643,5715162,0.371231
3,DRR121401,1311236,4546672,0.288395
4,ERR10695318,123175020,143348680,0.859269


In [47]:
join_g = join_g.rename(columns={"biome3": "biome"})
join_g.head()

,sample_id,ref_f_unweighted,ref_f_weighted,n_matches,biome
0,DRR014782,0.733621,0.949541,12389,Host-associated:Mammals:Respiratory system
1,DRR014785,0.728105,0.946977,13328,Host-associated:Mammals:Respiratory system
2,DRR019499,0.268765,0.259064,21126,Host-associated:Insecta:Digestive system
3,DRR042462,0.901071,0.965085,3213,Host-associated:Human:Digestive system
4,DRR042463,0.815821,0.937989,4229,Host-associated:Human:Digestive system


In [48]:
fmg_subset = fmg.merge(
    grist[['sample_id']],
    on='sample_id',
    how='inner'
)

In [49]:
# Normalize join keys
join_g = join_g.rename(columns={"accession": "sample_id"})

# Merge step-by-step
merged = (
    fmg_subset
    .merge(grist, on="sample_id", how="left", suffixes=("_fmg", "_grist"))
    .merge(fmap, on="sample_id", how="left")
    .merge(join_g, on="sample_id", how="left")
)

# -------------------------
# Optional: reorder columns for clarity
# -------------------------
ordered_cols = [
    "sample_id",

    # FMG
    "n_match_genomes_fmg",
    "avg_f_match",
    "avg_median_abund",
    "n_unique_weighted",

    # genome-grist
    "n_match_genomes",
    "avg_f_covered_bp",
    "avg_effective_coverage",

    # mapping summary
    "total_mapped_reads",
    "total_n_reads",
    "f_mapped_reads",

    # reference-level / biome
    "ref_f_weighted",
    "n_matches",
    "biome",
]

# keep only existing columns (in case some are missing)
ordered_cols = [c for c in ordered_cols if c in merged.columns]
merged = merged[ordered_cols]

# -------------------------
# Write output
# -------------------------
merged.sort_values("sample_id").to_csv(OUT, index=False)

print(f"Wrote {OUT} ({merged.shape[0]} samples)")


Wrote fmg_grist_table.csv (83 samples)


In [50]:
df = pd.read_csv("fmg_grist_table.csv")
df

,sample_id,n_match_genomes_fmg,avg_f_match,avg_median_abund,n_unique_weighted,n_match_genomes,avg_f_covered_bp,avg_effective_coverage,total_mapped_reads,total_n_reads,f_mapped_reads,ref_f_weighted,n_matches,biome
0,DRR014782,13557,0.019701,1.880910,658191,735,0.283430,5.906585,75612507,83531900,0.905193,0.949541,12389,Host-associated:Mammals:Respiratory system
1,DRR014785,14609,0.020887,1.879697,730806,892,0.272454,5.303160,83163638,91931356,0.904628,0.946977,13328,Host-associated:Mammals:Respiratory system
2,DRR121397,19375,0.006433,1.228387,85955,234,0.131079,1.655980,2121643,5715162,0.371231,0.648576,17436,Host-associated:Plants:Root
3,DRR121401,18383,0.005880,1.141843,65996,175,0.121921,1.409490,1311236,4546672,0.288395,0.601657,16482,Host-associated:Plants:Root
4,ERR10695318,14807,0.005826,6.803269,1732695,57,0.265579,126.762150,123175020,143348680,0.859269,0.877064,13000,Engineered:Food production:Dairy products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,SRR7299214,28773,0.008495,1.931290,215684,351,0.276998,3.056625,6705137,16634768,0.403080,0.566850,26105,Engineered:Solid waste:Landfill
79,SRR7610133,15970,0.005793,1.618910,92171,13,0.817984,18.349528,3058996,5994846,0.510271,0.526295,14140,Host-associated:Fish:Digestive system
80,SRR9109404,23035,0.006760,1.937031,387873,90,0.415485,15.212256,25075735,38762198,0.646912,0.756233,20680,Host-associated:Fish:Digestive system
81,SRR9182859,60347,0.010038,3.139651,1424535,739,0.317031,17.608389,104629074,162463680,0.644015,0.644513,52569,Host-associated:Invertebrates:Cnidaria
